This notebook covers the topic of quantum neural networks in real life. Code examples are presented for the quantum neural network topics related to quantum artificial neural networks (ANNs), quantum associative memory, quantum dots, quantum generative adversarial networks (GANs), and quantum random access memory.

**Quantum ANN** 

In this section, we will look at quantum artificial neural networks. Quantum neural networks are the quantum equivalent of neural networks, based on quantum mechanics principles. Quantum neural networks are based on quantum information processing and brain quantum effects. Quantum artificial neural networks are used for pattern classification algorithms.

A quantum neural network uses a quantum dot molecule and optical photons in the model. The temporal model of a QNN is modeled by the mathematical equations 
of the virtual neurons. The number of virtual neurons in a QNN is determined by the temporal discretization parameters. A new model was created by Wichita State University that is based on the spatial model of QNN. The spatial model uses the array of quantum dot molecules. Ron Chrisley, who was based in the University of Sussex, has proposed a different approach related to the network weights that are dependent on the position of silts. A quantum neural network that is based on neurocomputing is used for pattern recognition in real life. [Bhagvan Kommadi]

In [ ]:
# Quantum Neural Network (QNN)

import strawberryfields
import pennylane as qnnL
from pennylane import numpy as numcal
from pennylane.optimize import AdamOptimizer as AdamO

try:
    device = qnnL.device('strawberryfields.fock', wires=1, cutoff_dim=10)    
except:
    print("please install the package strawberryfields")


def GetQNNLayer(varr):
    """ One layer of the quantum neural network
    """
    qnnL.Rotation(varr[0], wires=0)
    qnnL.Squeezing(varr[1], 0., wires=0)
    qnnL.Rotation(varr[2], wires=0)

    qnnL.Displacement(varr[3], 0., wires=0)

    qnnL.Kerr(varr[4], wires=0)


@qnnL.qnode(device)
def GetQNN(vars, xcoor=None):
    """ Returns The quantum neural network 
    """

    qnnL.Displacement(xcoor, 0., wires=0)

    for var in vars:
        GetQNNLayer(var)

    return qnnL.expval.X(0)


def GetQNNSquareLoss(labelValues, predictionValues):
    """ Get the Square loss function
    """
    lossValue = 0
    for label, prediction in zip(labelValues, predictionValues):
        lossValue = lossValue + (label - prediction) ** 2
    lossValue = lossValue / len(labelValues)

    return lossValue


def GetQNNCost(variables, featureValues, labelValues):
    """ Minimizing Cost function 
    """
    
    predictions = [GetQNN(variables, xcoor=x) for x in featureValues]
    
    return GetQNNSquareLoss(labelValues, predictions)

sincurvevalues = numcal.loadtxt("sin_curve_values.txt")
xcoordinate= sincurvevalues[:, 0]
ycoordinate = sincurvevalues[:, 1]

numcal.random.seed(0)
layer_recount = 4
variable_initial = 0.05 * numcal.random.randn(layer_recount, 5)

optimizer = AdamO(0.01, beta1=0.9, beta2=0.999)


variable = variable_initial

for iteration in range(500):
    var_cost = optimizer.step(lambda v: GetQNNCost(v, xcoordinate, ycoordinate), variable)
    
    print("Iteration value: {:5d} | Cost is: {:0.7f} ".format(iteration + 1, GetQNNCost(var_cost, xcoordinate, ycoordinate)))

**Quantum Generative Adversarial Network Algorithms**.

Quantum GAN: In classical machine learning, generative adversarial networks are a tool. A generator is used to create statistics for data. It tries to mimic the current data set. A discriminator differentiates between the real and fake data. The learning process for a generator and a discriminator converges. The convergence happens when the generator generates the statistics. At the same time, the discriminator cannot differentiate between the real and generated data.

The quantum generative adversarial networks (QGANs) are generative adversarial
networks using quantum processors. Let’s look at an implementation of a quantum generative adversarial network. The implementation is based on two quantum circuits: the generator and the other discriminator. [Bhagvan Kommadi]

In [ ]:
# Quantum Generative Adversarial Networks (QGANs)

import pennylane as qganPenny
from pennylane import numpy as numcal
from pennylane.optimize import GradientDescentOptimizer as GDO

device = qganPenny.device('default.qubit', wires=3)


def GetQGANReal(phi, theta, omega):
    qganPenny.Rot(phi, theta, omega, wires=0)


def GetQGANGenerator(wireArray):
    qganPenny.RX(wireArray[0], wires=0)
    qganPenny.RX(wireArray[1], wires=1)
    qganPenny.RY(wireArray[2], wires=0)
    qganPenny.RY(wireArray[3], wires=1)
    qganPenny.RZ(wireArray[4], wires=0)
    qganPenny.RZ(wireArray[5], wires=1)
    qganPenny.CNOT(wires=[0,1])
    qganPenny.RX(wireArray[6], wires=0)
    qganPenny.RY(wireArray[7], wires=0)
    qganPenny.RZ(wireArray[8], wires=0)


def GetQGANDiscriminator(wireArray):
    qganPenny.RX(wireArray[0], wires=0)
    qganPenny.RX(wireArray[1], wires=2)
    qganPenny.RY(wireArray[2], wires=0)
    qganPenny.RY(wireArray[3], wires=2)
    qganPenny.RZ(wireArray[4], wires=0)
    qganPenny.RZ(wireArray[5], wires=2)
    qganPenny.CNOT(wires=[1,2])
    qganPenny.RX(wireArray[6], wires=2)
    qganPenny.RY(wireArray[7], wires=2)
    qganPenny.RZ(wireArray[8], wires=2)


@qganPenny.qnode(device)
def GetQGANRealDiscCircuit(phi, theta, omega, discWeights):
    GetQGANReal(phi, theta, omega)
    GetQGANDiscriminator(discWeights)
    return qganPenny.expval.PauliZ(2)


@qganPenny.qnode(device)
def GetQGANDiscCircuit(genWeights, discWeights):
    GetQGANGenerator(genWeights)
    GetQGANDiscriminator(discWeights)
    return qganPenny.expval.PauliZ(2)


def GetQGANRealTrue(discWeights):
    trueDiscriminatorOutput = GetQGANRealDiscCircuit(phi, theta, omega, discWeights)
    probabilityRealTrue = (trueDiscriminatorOutput + 1) / 2
    return probabilityRealTrue


def GetQGANFakeTrue(genWeights, discWeights):
    fakeDiscriminatorOutput = GetQGANDiscCircuit(genWeights, discWeights)
    probabilityFakeTrue = (fakeDiscriminatorOutput + 1) / 2
    return probabilityFakeTrue 


def GetQGANDiscriminatorCost(discWeights):
    cost = GetQGANFakeTrue(genWeights, discWeights) - GetQGANRealTrue(discWeights) 
    return cost


def GetQGANGeneratorCost(genWeights):
    return -GetQGANFakeTrue(genWeights, discWeights)


phi = numcal.pi / 6
theta = numcal.pi / 2
omega = numcal.pi / 7

numcal.random.seed(0)
epsValue = 1e-2
genWeights = numcal.array([numcal.pi] + [0] * 8) + numcal.random.normal(scale=epsValue, size=[9])
discWeights = numcal.random.normal(size=[9])

gdo = GDO(0.1)

print("Training the discriminator ")
for iteration in range(50):
    discriminator_weights = gdo.step(GetQGANDiscriminatorCost, discWeights) 
    discriminator_cost = GetQGANDiscriminatorCost(discriminator_weights)
    if iteration % 5 == 0:
        print("Iteration num {}: discriminator cost is = {}".format(iteration+1, discriminator_cost))

print(" discriminator - real true: ", GetQGANRealTrue(discWeights))
print("discriminator - fake true: ", GetQGANFakeTrue(genWeights, discWeights))

print("Training the generator.")


for iteration in range(200):
    generator_weights = gdo.step(GetQGANGeneratorCost, genWeights)
    generator_cost = -GetQGANGeneratorCost(generator_weights)
    if iteration % 5 == 0:
        print("Iteration num {}: generator cost is = {}".format(iteration, generator_cost))

print("discriminator - real true: ", GetQGANRealTrue(discWeights))
print("discriminator -  fake true: ", GetQGANFakeTrue(genWeights, discWeights))

print("Cost is: ", GetQGANDiscriminatorCost(discriminator_weights))